In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

C:\Users\elrii\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
input_dir = r'./input/'
output_dir = r'./output/'

In [60]:
dfa = pd.read_sas(input_dir+'AUX_J_2017.XPT')
dfq = pd.read_sas(input_dir+'AUQ_J_2017.XPT')
dfd = pd.read_sas(input_dir+'DEMO_J_2017.XPT') #CHANGE

In [61]:
#dfq.to_excel(output_dir+"hi.xlsx")

In [62]:
dfa = dfa[['SEQN', 'AUXU500R','AUXU1K2R','AUXU2KR','AUXU3KR',
            'AUXU4KR','AUXU6KR','AUXU8KR','AUXU500L',
            'AUXU1K2L','AUXU2KL','AUXU3KL','AUXU4KL','AUXU6KL',
            'AUXU8KL']].replace(666,120)

audiogram_col = ['AUXU500R','AUXU1K2R','AUXU2KR','AUXU3KR',
            'AUXU4KR','AUXU6KR','AUXU8KR','AUXU500L',
            'AUXU1K2L','AUXU2KL','AUXU3KL','AUXU4KL','AUXU6KL',
            'AUXU8KL']
audiogram_right = audiogram_col[:7]
audiogram_left = audiogram_col[7:]

dfa = dfa[~dfa[audiogram_col].isin([888]).any(axis=1)].dropna().astype(int)
dfa = dfa[(dfa[audiogram_right].sum(axis=1) >= 140) & (dfa[audiogram_right].sum(axis=1) >= 140)].reset_index(drop=True)
dfa.shape

(806, 15)

In [63]:
a = 'AUQ054'
b = 'AUQ191'
c = 'AUQ330'
dfq = dfq[['SEQN', a, b, c]] #CHECK
dfq=dfq.dropna(how='any').reset_index(drop=True)
dfq = dfq[dfq[a] <= 6].reset_index(drop=True)
dfq = dfq[dfq[b] <= 2].reset_index(drop=True)
dfq = dfq[dfq[c] <= 2].reset_index(drop=True)
new_column_names = {a: 'Q1', b: 'Q2', c: 'Q3'}
dfq.rename(columns=new_column_names, inplace=True)
dfq.shape

(1545, 4)

In [64]:
military = 'DMQMILIZ'
dfd = dfd[['SEQN', 'RIAGENDR','RIDAGEYR', military]] #1 = male, 2 = female

thresholds = [0, 25, 35, 45, 55, dfd['RIDAGEYR'].max()] 
labels = [1, 2, 3,4,5]
dfd['RIDAGEYR'] = pd.cut(dfd['RIDAGEYR'], 
                         bins=thresholds, labels=labels, right=True)
dfd = dfd.dropna(subset=[military]).reset_index(drop=True)
dfd = dfd[dfd[military] <= 2].reset_index(drop=True)
dfd.rename(columns={military: "Military"}, inplace=True)
dfd.shape

(6001, 4)

In [65]:
dfV = pd.merge(pd.merge(dfd, dfa, on='SEQN'), dfq).astype(int)
print(dfV.shape)
dfV.to_excel(output_dir+"NHANES_2017-2018.xlsx") #CHANGE

(766, 21)


In [66]:
dfV

,SEQN,RIAGENDR,RIDAGEYR,Military,AUXU500R,AUXU1K2R,AUXU2KR,AUXU3KR,AUXU4KR,AUXU6KR,...,AUXU500L,AUXU1K2L,AUXU2KL,AUXU3KL,AUXU4KL,AUXU6KL,AUXU8KL,Q1,Q2,Q3
0,93709,2,5,2,5,20,30,25,25,30,...,15,20,30,20,20,25,25,1,2,2
1,93715,1,5,2,5,10,15,65,70,60,...,5,15,55,65,70,65,70,3,2,1
2,93727,1,5,2,5,-5,35,50,60,70,...,10,10,15,10,15,15,25,1,2,2
3,93732,1,5,2,15,10,40,55,75,85,...,10,10,30,55,70,60,70,3,2,1
4,93740,1,5,2,50,60,45,55,55,60,...,20,20,30,35,30,45,65,5,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
761,102890,1,5,1,25,25,70,65,75,85,...,35,30,75,70,90,95,120,4,1,1
762,102904,2,5,2,10,10,15,30,35,20,...,20,5,15,30,40,40,35,1,2,2
763,102910,1,5,2,15,5,10,30,15,35,...,15,10,10,40,40,25,20,2,2,1
764,102924,2,5,2,25,25,45,55,60,55,...,30,15,50,50,65,60,60,4,1,1
